# Make Small Data
Code in this notebook collects merged_data.csv, filters the variables that we chose, imputes missing data using earlier years, and calculates missingness by country and indicator. Produces small_data.csv, small_data_imputed.csv, indicator_coverage.csv, and country_coverage.csv

In [139]:
import pandas as pd
import numpy as np
import os

In [146]:
df = pd.read_csv('data/merged_data.csv')
variables = pd.read_csv('data/variables.csv')

In [152]:
# 0s vs missingness for migration
cols = ['iso', 'country_name', 'year']

migration = ['ims_both_sex', 'estimated_refugee_stock_incl_asylum_seekers_both_sexes', 'disaster_stock_displacementr_raw', 'conflict_stock_displacement_raw']

climate = ['CCH', 'AIR', 'hdi_value']

# no CPI corruption data, replaced with WWGI corruption
governance = ['value.Rule of Law: Estimate', 'value.Government Effectiveness: Estimate', 'value.Control of Corruption: Estimate']

ys = ['P1: State Legitimacy', 'D_avg', 'D12']

controls = ['GDP per capita (constant 2015 US$)', 'value.Gini index (World Bank estimate)']

all_cols = cols + migration + climate + governance + controls + ys

In [170]:
small_df = df[all_cols]
small_vars = variables[variables.variable.isin(all_cols)]
small_vars

,file_name,data_source,variable,min_year,max_year
0,wwgi_clean.csv,WWGI,value.Control of Corruption: Estimate,2011.0,2020.0
1,wwgi_clean.csv,WWGI,value.Government Effectiveness: Estimate,2011.0,2020.0
4,wwgi_clean.csv,WWGI,value.Rule of Law: Estimate,2011.0,2020.0
8,IDMC_Conflict and Disaster Total.csv,IDMC,conflict_stock_displacement_raw,2008.0,2020.0
14,IDMC_Conflict and Disaster Total.csv,IDMC,disaster_stock_displacementr_raw,2008.0,2020.0
16,"UN HDI, Environment Pillar.csv",UN_HDI,hdi_value,2010.0,2019.0
19,epi.csv,Yale EPI,AIR,2010.0,2020.0
25,epi.csv,Yale EPI,CCH,2010.0,2020.0
71,UNDESA_clean.csv,UNDESA,ims_both_sex,2020.0,2020.0
81,UNDESA_clean.csv,UNDESA,estimated_refugee_stock_incl_asylum_seekers_bo...,2020.0,2020.0


## Create a DataFrame with the latest data available for each indicator

In [154]:
dfs = []
flag = True
for year in small_vars.max_year.unique():
    var_list = list(small_vars[small_vars['max_year']==year].variable)
    c_df = small_df.loc[small_df.year==year, cols + var_list]
    c_df.drop(['year'], inplace=True, axis=1)
    if flag:
        first_df = c_df
        flag = False
        continue
    first_df = first_df.merge(c_df, how='outer', on=['iso', 'country_name'])
first_df.reset_index(drop=True, inplace=True)
first_df.head()

,iso,country_name,value.Control of Corruption: Estimate,value.Government Effectiveness: Estimate,value.Rule of Law: Estimate,conflict_stock_displacement_raw,disaster_stock_displacementr_raw,AIR,CCH,ims_both_sex,estimated_refugee_stock_incl_asylum_seekers_both_sexes,GDP per capita (constant 2015 US$),D12,D_avg,hdi_value,value.Gini index (World Bank estimate),P1: State Legitimacy
0,NPL,Nepal,-0.577751,-0.944426,-0.490970,0.0,28000.0,14.6,23.7,487564.0,19634.0,1028.463277,3.5,3.9,0.602,NaN,6.4
1,AFG,Afghanistan,-1.475405,-1.523115,-1.807697,3546858.0,1117000.0,17.7,22.2,144098.0,72479.0,529.741210,1.5,2.8,0.511,NaN,8.7
2,ARM,Armenia,0.031597,-0.124236,-0.083223,800.0,2700.0,36.3,46.7,190349.0,18158.0,4021.046264,NaN,NaN,0.776,29.9,6.9
3,AUS,Australia,1.665165,1.619163,1.651188,0.0,5100.0,98.2,70.4,7685860.0,154129.0,58043.580660,NaN,NaN,0.944,NaN,0.5
4,AZE,Azerbaijan,-1.048197,-0.170233,-0.691043,735455.0,0.0,24.9,48.6,252228.0,1288.0,5083.381639,NaN,NaN,0.756,NaN,9.1


In [191]:
df.loc[df.iso=='AUS', ['year', 'value.Gini index (World Bank estimate)']]

,year,value.Gini index (World Bank estimate)
3,2011,NaN
50,2012,NaN
97,2013,NaN
144,2014,34.4
191,2015,NaN
238,2016,NaN
285,2017,NaN
332,2018,NaN
379,2019,NaN
426,2020,NaN


## Impute missing data with the most recent available data

In [213]:
sorted_df = df.sort_values('year', ascending=False)

imputed_df = first_df.copy(deep=True)
for i, row in imputed_df.iterrows():
    country = row['iso']
    for k, v in row.items():
        if pd.isna(v) and k not in ys:
            try: 
                filling = df.loc[(df.iso==country) & (~df[k].isna()), [k]].iat[0, 0]
                recent_year = df.loc[(df.iso==country) & (~df[k].isna()), ['year']].iat[0, 0]
                imputed_df.at[i, k] = filling
                print(f"imputed {country}: {k} with {filling} ({recent_year})")
            except IndexError:
                continue

imputed NPL: value.Gini index (World Bank estimate) with 32.8 (2010)
imputed AUS: value.Gini index (World Bank estimate) with 34.4 (2014)
imputed BGD: value.Gini index (World Bank estimate) with 32.4 (2016)
imputed BTN: value.Gini index (World Bank estimate) with 38.8 (2012)
imputed BRN: conflict_stock_displacement_raw with 0.0 (2014)
imputed BRN: disaster_stock_displacementr_raw with 0.0 (2014)
imputed COK: value.Control of Corruption: Estimate with -0.2318449 (2011)
imputed COK: value.Government Effectiveness: Estimate with -0.9730545 (2011)
imputed COK: value.Rule of Law: Estimate with -0.8820833 (2011)
imputed COK: conflict_stock_displacement_raw with 0.0 (2016)
imputed COK: disaster_stock_displacementr_raw with 0.0 (2016)
imputed COK: AIR with 100.0 (2012)
imputed FSM: conflict_stock_displacement_raw with 0.0 (2012)
imputed FSM: disaster_stock_displacementr_raw with 0.0 (2012)
imputed FSM: value.Gini index (World Bank estimate) with 40.1 (2013)
imputed FJI: value.Gini index (World

## Get Coverage by Country

In [214]:
missing_df = ~(imputed_df.iloc[:, 2:].isna())
missing_df.insert(0, 'country_name', imputed_df['country_name'])
missing_df.insert(0, 'iso', imputed_df['iso'])
missingness = missing_df.iloc[:, 2:].sum(axis=1) / missing_df.iloc[:, 2:].shape[1]
missing_df.insert(2, 'completeness', missingness)
missing_df.sort_values(by='completeness', inplace=True)
missing_df

,iso,country_name,completeness,value.Control of Corruption: Estimate,value.Government Effectiveness: Estimate,value.Rule of Law: Estimate,conflict_stock_displacement_raw,disaster_stock_displacementr_raw,AIR,CCH,ims_both_sex,estimated_refugee_stock_incl_asylum_seekers_both_sexes,GDP per capita (constant 2015 US$),D12,D_avg,hdi_value,value.Gini index (World Bank estimate),P1: State Legitimacy
48,TWN,"Taiwan, Province of China",0.266667,False,False,False,True,True,True,True,False,False,False,False,False,False,False,False
28,NIU,Niue,0.333333,True,True,True,False,False,True,False,True,False,False,False,False,False,False,False
47,SGP,Singapore,0.400000,False,False,False,False,False,True,True,True,False,True,False,False,True,False,True
9,COK,Cook Islands,0.466667,True,True,True,True,True,True,False,True,False,False,False,False,False,False,False
43,TUV,Tuvalu,0.666667,True,True,True,True,True,True,False,False,False,True,True,True,False,True,False
26,NRU,Nauru,0.666667,True,True,True,False,False,True,False,True,True,True,True,True,False,True,False
7,BRN,Brunei Darussalam,0.733333,True,True,True,True,True,True,True,True,False,True,False,False,True,False,True
18,KIR,Kiribati,0.733333,True,True,True,True,True,True,True,True,False,True,True,True,False,False,False
13,HKG,Hong Kong,0.733333,True,True,True,True,True,True,True,True,True,True,False,False,True,False,False
30,PLW,Palau,0.800000,True,True,True,True,True,True,True,True,False,True,True,True,True,False,False


## Get Coverage by Indicator

In [224]:
missing = list(missing_df.iloc[:, 3:].sum(axis=0) / missing_df.iloc[:, 2:].shape[0])
missing = [[i] for i in missing]
missing_cols = pd.DataFrame((dict(zip(list(missing_df.columns)[3:], missing ))))
missing_cols = missing_cols.T.reset_index()
missing_cols.columns = ['variable', 'coverage']
missing_cols

,variable,coverage
0,value.Control of Corruption: Estimate,0.959184
1,value.Government Effectiveness: Estimate,0.959184
2,value.Rule of Law: Estimate,0.959184
3,conflict_stock_displacement_raw,0.938776
4,disaster_stock_displacementr_raw,0.938776
5,AIR,1.000000
6,CCH,0.918367
7,ims_both_sex,0.959184
8,estimated_refugee_stock_incl_asylum_seekers_bo...,0.591837
9,GDP per capita (constant 2015 US$),0.938776


## Save Data

In [225]:
df.to_csv('data/cleaned-datasets/final/merged_data.csv', index=False)
first_df.to_csv('data/cleaned-datasets/final/small_data.csv', index=False)
imputed_df.to_csv('data/cleaned-datasets/final/small_data_imputed.csv', index=False)
missing_cols.to_csv('data/cleaned-datasets/final/indicator_coverage.csv', index=False)
missing_df.to_csv('data/cleaned-datasets/final/country_coverage.csv', index=False)